# Data preparations

This notebooks tries to define the process of defining 

In [ ]:
# Base libraries
import sys
import pandas as pd
import matplotlib
import numpy as np
import scipy as sp
import sklearn

#Garbage Collection Library
import gc

# SQL Alchemy
from sqlalchemy import create_engine
from sqlalchemy.engineport SafeLoader

#SQL functionality
import duckdb

# For SKLearn import
from sklearn.pipeline import *
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
import joblib

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
'''#files are loaded from config folder'''
from config_loader import params, model_params

version = params['version']
print(f'Code version: {version}')

In [ ]:
model_params['categories'][0]

In [ ]:
connect_url = url.URL(
    params['driver'],
    username=params['username'],
    password=params['password'],
    host=params['host'],
    port=params['port'],
    query=dict(service_name=params['service_name']))
engine = create_engine(connect_url)
connection = engine.connect()

In [ ]:
'''#In order to make readable code, variables are here
    #In order to work properly:
    #dataset = model_params['datasource']'''
def import_data():
    dataset = 'sample.csv'
    data = pd.read_csv('../../data/{0}'.format(dataset))
    return data

In [ ]:
df = import_data()
df.head()

In [ ]:
'''
    Both functions return the same dataframe.
    
    SQL way is the fastest way to get a production dataset. So if you can't write transformations as a SQL sentence, it's THINKING TIME
     - Maybe, this is not the place to write this code and this is a transformation pipeline that need's python language instead sql language. Then create a new notebook and define
       new stage for your project. Read the documentation and look for 'Pipeline enrichment' section
     - Maybe, this is the place to write this code, but you need help with the SQL sentence. Call us
     
    Remember, transformation starts and ends with a dataset located in a database. This is not part of model is part of variables creation.
    
    Let' go to win-win scenario. 
        -> If your transformation process is supported on SQL language, PRODUCTION TIME decrease dramatically. Transformations works as you say not as we understand that you say.
           Understanding and interpretation are enemies of Time to Market.
        -> If your tranformation process is supported on SQL language, not NOW, but we are working on comunity datasets where you could find a lot of calculated variables shared 
           by other DS.
        -> Don't worry if you need help, this is our work.
    
'''
def add_category_python(df):
    """
    Adds category field given certain conditions
    """
    df['cat'] = 0
    df['cat'] = np.where((
        (df['pk_cod_clop']=='27')|
        (df['pk_cod_clop'].isin([41,91])))|
                         (df['pk_cod_clop']=='28')&(df['pk_cod_subclop'].between(11,13)), 1, 
                         np.where((df['pk_cod_clop']=='70')&(df['pk_cod_subclop'].isin([23])), 2, 
                                  np.where((df['pk_cod_clop']=='19')&(df['pk_cod_subclop'].isin([1])), 3, 0)))
    return df

def add_category_sql(df):
    """
    Adds category field given certain conditions
    """
    sql = "SELECT df.*, CASE WHEN ((PK_COD_CLOP = 27) OR (PK_COD_CLOP IN (41, 91)) OR PK_COD_CLOP = 81 AND PK_COD_SUBCLOP IN (11, 12, 13)) THEN 1 \
                            WHEN (PK_COD_CLOP = 70 AND PK_COD_SUBCLOP IN (23)) THEN 2 \
                            WHEN (PK_COD_CLOP IN (19) AND PK_COD_SUBCLOP IN (1)) THEN 3 \
                            ELSE 0 END AS CAT \
                        FROM df"
    return duckdb.query(sql).to_df()


In [ ]:
'''
  If you want to compare output dataframes from 'add_category_python' and 'add_category_sql', this function makes this comparation for you.
'''
# Expected dataframe
df_exp = import_data()
df_exp = add_category_python(df_exp)

# Obtained by the new function
df_out = add_category_sql(df)
df_out.head()

In [ ]:
pd.testing.assert_frame_equal(df_out, df_exp, check_dtype=False)